下载数据

In [1]:
# google drive link
!gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
!gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link
#!wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
#!wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

/opt/anaconda3/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-
To: /Volumes/zkj/personal/ml/lhy/Lhy_Machine_Learning/2023 ML/我的作页/HW1 Regression/covid_train.csv
100%|██████████████████████████████████████| 2.16M/2.16M [00:00<00:00, 5.67MB/s]
/opt/anaconda3/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1
To: /Volumes/zkj/personal/ml/lhy/Lhy_Machine_Learning/2023 ML/我的作页/HW1 Regression/covid_test.csv
100%|████████████████████████████████████████| 638k/638k [00:00<00:00, 2.44MB/s]


Import packages

In [11]:
import pandas as pd
import numpy as np

import os
import math

from tqdm import tqdm
import csv

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

Config

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-5,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}




utils

In [4]:
def same_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


数据集类


In [5]:
class COVID19Dataset(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.x[idx], self.y[idx]
        else:
            return self.x[idx]
    
    
def train_valid_split(data_set, valid_ratio, seed):
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))

    return np.array(train_set), np.array(valid_set)

dataLoader


一个Epoch
├── Batch 1 (样本1-32)
├── Batch 2 (样本33-64)
├── ...
└── Batch n (样本...-N)

In [30]:
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

y_train, y_valid = train_data[:,-1], valid_data[:,-1]
x_train, x_valid, x_test = train_data[:, :-1], valid_data[:, :-1], test_data[:, :]

print(f"""y_train size: {y_train.shape} 
y_valid size: {y_valid.shape} """)

print(f"""x_train size: {x_train.shape} 
      x_test size: {x_test.shape} 
x_vaild size: {x_valid.shape} """)

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2408, 89) 
valid_data size: (601, 89) 
test_data size: (997, 88)
y_train size: (2408,) 
y_valid size: (601,) 
x_train size: (2408, 88) 
      x_test size: (997, 88) 
x_vaild size: (601, 88) 


模型

In [7]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),  # 第一层全连接
            nn.ReLU(),                 # 激活函数
            nn.Linear(16, 8),          # 第二层全连接
            nn.ReLU(),                 # 激活函数
            nn.Linear(8, 1)            # 输出层
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

训练

In [20]:
def trainer(train_loader, valid_loader, model, config, device):
    # loseFuction
    criterion = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7)

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train()

        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1

            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        # Set your model to evaluation mode.
        model.eval()
        loss_record = []

        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

model = My_Model(input_dim=x_train.shape[1]).to(device)
trainer(train_loader, valid_loader, model, config, device)


Epoch [4638/5000]: Train loss: 1.6422, Valid loss: 1.3720


Epoch [4639/5000]:   0%|          | 0/10 [00:00<?, ?it/s, loss=1.18]

Epoch [4639/5000]: Train loss: 1.5068, Valid loss: 2.3663


Epoch [4640/5000]: Train loss: 1.7943, Valid loss: 1.7365


Epoch [4641/5000]: Train loss: 1.4104, Valid loss: 1.6034


...

Epoch [4868/5000]: Train loss: 1.6870, Valid loss: 2.0810


Epoch [4869/5000]: Train loss: 1.5918, Valid loss: 1.3176


Epoch [4870/5000]: Train loss: 1.4861, Valid loss: 1.7943


Epoch [4871/5000]: Train loss: 2.5413, Valid loss: 1.2637


Epoch [4872/5000]: Train loss: 1.5009, Valid loss: 1.3461

Model is not improving, so we halt the training session.


tensorboard

In [ ]:

%reload_ext tensorboard
%tensorboard --logdir=./runs/

Testing

In [31]:
def predict(test_loader, model, device):
    model.eval()
    preds = []

    for x in tqdm(test_loader):
        print(x.shape)
        x = x.to(device)
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    print('preds old:', preds)
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
print('preds new:', preds)

save_pred(preds, 'pred.csv')

100%|██████████| 4/4 [00:00<00:00, 309.89it/s]

torch.Size([256, 88])
torch.Size([256, 88])
torch.Size([256, 88])
torch.Size([229, 88])
preds old: [tensor([38.5621,  2.5433, 26.1439, 29.0616, 48.1557,  5.1093,  5.7421, 21.5147,
         8.0090,  9.5791, 15.3837, 15.3615, 29.3582,  9.1162, 16.2119, 26.0473,
        47.9161, 11.2454, 38.1034, 16.3968, 30.8295, 23.3543,  8.0986, 15.3800,
        14.6043,  9.7130, 18.1604, 18.7588,  1.8766, 37.4680,  2.0473, 40.8289,
        37.8195, 22.8845, 13.0936, 44.5287, 12.5183, 14.8727, 38.3214, 21.4980,
         4.5633, 28.6263, 21.1655, 22.0703, 11.1800,  5.6679,  7.5470, 15.5970,
        16.2295,  3.6894, 17.4392, 22.8185, 12.0286,  5.8452, 43.2394, 17.2700,
        21.8091,  8.2454, 23.7013, 14.2015, 12.0533, 20.6256, 18.2986, 17.2277,
        37.0289, 12.8241, 29.6324,  9.6024, 36.5352, 26.5353,  7.4703, 25.6813,
        10.8000,  8.9824, 27.3009,  5.8165,  3.4075, 14.4514,  2.5283, 17.5487,
        25.7254,  9.8080, 45.6957,  7.2063, 29.8789, 16.2325,  6.2309, 41.2405,
        13.8458, 16.